In [1]:
import glob
import tarfile
import xmltodict
import json

In [2]:
def openTarFile ( filename ):

    with tarfile.open(filename, "r:gz") as file:
        # don't use file.members as it's 
        # not giving nested files and folders
        
        dictdata = []

        for member in file:        
            if member.name.endswith ('xml'):
                data = file.extractfile(member.name).read()
                dictdata.append(xmltodict.parse(data))
        return dictdata


In [206]:
validForm = ['F01','F02']

keysToSaveMain = ['@DOC_ID']
keysTranslateMain = {'@DOC_ID':'doc_id'}

keysToSaveMainTechnicalSection = ['RECEPTION_ID']
keysTranslateTechnicalSection = {'RECEPTION_ID':'reception_id'}


def translateKeys ( tedDoc ):
    data = {}
    
    formdata = {}
    formdata['FORM_ID'] = list (tedDoc['TED_EXPORT']['FORM_SECTION'].keys())[0]
        

    data = {keysTranslateMain[key] : tedDoc['TED_EXPORT'][key] for key in keysToSaveMain }

    #si queremos un diccionarion con otro diccionario para cada sección:
    #data['TECHNICAL_SECTION'] = {keysTranslateTechnicalSection[key]: tedDoc['TED_EXPORT']['TECHNICAL_SECTION'][key] for key in keysTranslateTechnicalSection }

    #si queremos un sólo nivel:
    data2 = {keysTranslateTechnicalSection[key]: tedDoc['TED_EXPORT']['TECHNICAL_SECTION'][key] for key in keysTranslateTechnicalSection }    
    data.update (data2)

    data.update (formdata)

    #si es una lista de valores es que se trata de una traduccion, un idioma en cada uno
    data['EN_TRANSLATION'] = False            
    if type (tedDoc['TED_EXPORT']['FORM_SECTION'][formdata['FORM_ID']]) == list:            
        for value in tedDoc['TED_EXPORT']['FORM_SECTION'][formdata['FORM_ID']]:
            if value['@LG'] == 'EN' and value['@CATEGORY'] == 'TRANSLATION':
                data['EN_TRANSLATION'] = True
                data['TITLE_EN'] = value['OBJECT_CONTRACT']['TITLE']['P']
                data['TEST'] = value
            if  value['@CATEGORY'] == 'ORIGINAL':
                data['ORIGINAL_LG'] = value['@LG']
                data['TEST'] = value
                #data['TRANSLATE_TITLE'] = value['OBJECT_CONTRACT']['TITLE']        
    else:
        data['ORIGINAL_LG'] = tedDoc['TED_EXPORT']['FORM_SECTION'][formdata['FORM_ID']]['@LG']
        #data['ORIGINAL_TITLE'] = tedDoc['TED_EXPORT']['FORM_SECTION'][formdata['FORM_ID']]['OBJECT_CONTRACT']['TITLE']
        data['TEST'] = tedDoc['TED_EXPORT']['FORM_SECTION'][formdata['FORM_ID']]

                

    
    return data

In [ ]:
year = 2015
path ='///export/data_ml4ds/IntelComp/Datasets/ted/xmls/'
filePattern = path + '*' + str(year) + '*.tar.gz'

totalResult = []

for file in glob.glob( filePattern ):
    print (file)
    dataFile = openTarFile (file)
    totalResult += [translateKeys (data) for data in dataFile]
    



In [207]:
###################################PRUEBAS 

file ='///export/data_ml4ds/IntelComp/Datasets/ted/pruebas/test.tgz'
file = '///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2022-03.tar.gz'
totalResult = []

print ('abriendo fichero')
dataFile = openTarFile (file)

print ('Filtrando formularios, en el original hay %s datos' % len(dataFile))
validForm = ['F01','F02']
dataFile = list(filter (lambda d: list (d['TED_EXPORT']['FORM_SECTION'].keys())[0][:3] in validForm, dataFile))
print ('nos quedamos con %s datos' % len (dataFile))

print ('convirtiendo datos')
totalResult += [translateKeys (data) for data in dataFile]




abriendo fichero
Filtrando formularios, en el original hay 59646 datos
nos quedamos con 20523 datos
convirtiendo datos


In [208]:
trans = list(filter (lambda d:d['EN_TRANSLATION']==False , totalResult))

In [210]:
trans[22]

{'doc_id': '145446-2022',
 'reception_id': '22-146640-001',
 'FORM_ID': 'F02_2014',
 'EN_TRANSLATION': False,
 'ORIGINAL_LG': 'BG',
 'TEST': OrderedDict([('@CATEGORY', 'ORIGINAL'),
              ('@FORM', 'F02'),
              ('@LG', 'BG'),
              ('LEGAL_BASIS', OrderedDict([('@VALUE', '32014L0024')])),
              ('CONTRACTING_BODY',
               OrderedDict([('ADDRESS_CONTRACTING_BODY',
                             OrderedDict([('OFFICIALNAME', 'ОБЩИНА ПАЗАРДЖИК'),
                                          ('NATIONALID', '000351736'),
                                          ('ADDRESS', 'бул. БЪЛГАРИЯ № 2'),
                                          ('TOWN', 'гр. Пазарджик'),
                                          ('POSTAL_CODE', '4400'),
                                          ('COUNTRY',
                                           OrderedDict([('@VALUE', 'BG')])),
                                          ('CONTACT_POINT',
                                        

In [140]:
#limpiamos los que no sean de los formularios que queremos:

totalResult = list(filter (lambda d: d != {}, totalResult))

In [141]:
lgfilter = list(filter (lambda d: d['EN_TRANSLATION'] == True, totalResult))

In [143]:


totalResult


[{'doc_id': '145861-2022',
  'reception_id': '22-146719-001',
  'FORM_ID': 'F01_2014',
  'EN_TRANSLATION': False,
  'ORIGINAL_LG': 'NL',
  'ORIGINAL_TITLE': 'Marktconsultatie levensmiddelen'},
 {'doc_id': '146236-2022',
  'reception_id': '22-148834-001',
  'FORM_ID': 'F02_2014',
  'EN_TRANSLATION': False,
  'ORIGINAL_LG': 'SL',
  'ORIGINAL_TITLE': 'Najem PET/CT skenerja s pripadajočo opremo, ureditvijo prostora ter preureditvijo čakalnice'},
 {'doc_id': '146134-2022',
  'reception_id': '22-149202-001',
  'FORM_ID': 'F02_2014',
  'EN_TRANSLATION': False,
  'ORIGINAL_LG': 'EL',
  'ORIGINAL_TITLE': '(Α.Δ. 13/22) Προμήθεια Ελαιολιπαντικών Λιπάνσεως Μέσων Εδάφους για Κάλυψη Αναγκών των Κλάδων των ΕΔ'},
 {'doc_id': '145431-2022',
  'reception_id': '22-147820-001',
  'FORM_ID': 'F01_2014',
  'EN_TRANSLATION': False,
  'ORIGINAL_LG': 'ES',
  'ORIGINAL_TITLE': 'Acondicionamiento y ampliación de gálibo en el túnel de Roda de Bará para la implantación del ancho estándar en el Corredor Mediterráne

In [92]:
totalResult

[{'doc_id': '145861-2022',
  'reception_id': '22-146719-001',
  'FORM_ID': 'F01_2014',
  'ORIGINAL_LG': 'NL'},
 {'doc_id': '146236-2022',
  'reception_id': '22-148834-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'SL'},
 {'doc_id': '146134-2022',
  'reception_id': '22-149202-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'EL'},
 {'doc_id': '145431-2022',
  'reception_id': '22-147820-001',
  'FORM_ID': 'F01_2014',
  'ORIGINAL_LG': 'ES'},
 {'doc_id': '146008-2022',
  'reception_id': '22-148045-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'PL'},
 {'doc_id': '145952-2022',
  'reception_id': '22-147626-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'PL'},
 {'doc_id': '145888-2022',
  'reception_id': '22-146839-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'IT'},
 {'doc_id': '145466-2022',
  'reception_id': '22-146894-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'DE'},
 {'doc_id': '145905-2022',
  'reception_id': '22-147236-001',
  'FORM_ID': 'F02_2014',
  'ORIGINAL_LG': 'PL'},
 

In [ ]:
'''
PRUEBA PARA HACER UN SOLO FICHERO NO EL BUCLE ANERIOR
year = 2017
file = '///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2017-08.tar.gz'
totalResult = []

dataFile = openTarFile (file)

totalResult = [translateKeys (data) for data in dataFile]
'''

In [ ]:
len(totalResult)
totalResult[0]
len(dataFile)


In [ ]:
from pyspark.sql import Row


df = spark.createDataFrame(totalResult)




In [ ]:
savePath = '/export/ml4ds/temporal/teddata/' + str(year)


df.write.mode("overwrite").parquet (savePath)

In [ ]:
df.select('doc_id').distinct().count()

In [ ]:
patata[0]['doc_id']